# Keyword Extraction (w/ BERT)

In [1]:
import tensorflow as tf

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertModel, BertForMaskedLM

import numpy as np
import pandas as pd
import random
import collections, codecs, re

import logging
logging.basicConfig(level=logging.INFO)

import spacy
import nltk    # stopword 목록 포함되어 있음
import string  # 특수문자 제거 위함
import csv # csv file 형태로 Dictionary 추출 위함
import en_core_web_sm
import re, regex

from gensim import corpora, models, similarities
from nltk.stem import WordNetLemmatizer # 표제어 추출 위함
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS # english_stop_word list 사용

nltk.download('averaged_perceptron_tagger')

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/dam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## 1. Prepare News Data

In [2]:
sample_data = pd.read_csv('../../02. Data/SKH_News Items_201601_20170120_175001.txt', encoding ='utf-8', sep = "|")
sample_data.head()

,Story Date Time,Headline,Story Body,Related RICs,Topics
0,12/31/2015 15:00:00,U.S. 30-YR FIXED RATE MORTGAGES 4.01 PCT DEC 3...,NaN,FMCC.OB,FIN US COFS MTG INT DBT WASH AGN BSVC FINS MCE...
1,12/31/2015 15:00:00,"TABLE-U.S. 30-year, 15-year mortgages",DEC 31 (REUTERS) - FREDDIE MAC <FMCC.OB> ...,FMCC.OB,FIN US COFS MTG INT DBT WASH AGN BSVC FINS MCE...
2,12/31/2015 15:00:03,US STOCKS-Indexes open lower on last day of year,* S&P 500 turns negative for year * Ap...,.DJI .INX .IXIC .SPX,STX US MKTREP REP MTGFX LEN AMERS RTRS
3,12/31/2015 15:00:09,ADVISORY-U.S./Canada daily earnings hits & mis...,Dec 31 (Reuters) - The table showing wheth...,NaN,CMPNY REP LEN US CA AMERS RES BACT AGA RTRS
4,12/31/2015 15:00:20,MEXICO CENTRAL BANK MINUTES SHOW DECISION TO ...,NaN,NaN,MX LATAM EMRG CEN MCE FRX GVD INT PLCY LEN RTRS


In [3]:
# Delete NaN Data Line
print("Origin_Data :", len(sample_data))
sample_data = sample_data.dropna()
print("After Delete :", len(sample_data))
sample_data

Origin_Data : 89767
After Delete : 21948


,Story Date Time,Headline,Story Body,Related RICs,Topics
1,12/31/2015 15:00:00,"TABLE-U.S. 30-year, 15-year mortgages",DEC 31 (REUTERS) - FREDDIE MAC <FMCC.OB> ...,FMCC.OB,FIN US COFS MTG INT DBT WASH AGN BSVC FINS MCE...
2,12/31/2015 15:00:03,US STOCKS-Indexes open lower on last day of year,* S&P 500 turns negative for year * Ap...,.DJI .INX .IXIC .SPX,STX US MKTREP REP MTGFX LEN AMERS RTRS
13,12/31/2015 15:01:27,UPDATE 2-Brazil's Itaú buys BTG Pactual's debt...,"(Adds quote, background) By Guillermo Par...",BBTG11.SA ITUB4.SA BPAC3.SA ITUB.N,LOA CDM FINS BANK INVS SFIN DFIN INVD DEAL1 EM...
27,12/31/2015 15:04:09,UPDATE 1-Saudi's market regulator refers suspe...,* CMA refers unnamed suspects to authoriti...,7020.SE,TCOM PHON WLES ACCI EMRG SA MEAST JUDIC FRAUD1...
38,12/31/2015 15:08:13,BUZZ-Prima BioMed Ltd: Roth Capital starts cov...,** Australian biotech's U.S.-listed shares...,PRR.AX,STX HOT HECA PHMR RCH US LEN BLR HEA CMPNY MRC...
40,12/31/2015 15:08:39,BUZZ-MXN-Banxico rate hike vote was unanimous,* Decision to hike rates 25bps (to 3.2...,LAN.SN,AIRL AIRS AMERS CL CMPNY EMRG FRX INDS INSI LA...
43,12/31/2015 15:10:13,"Nigerian naira down 10 pct yr/yr, stocks fall ...","LAGOS, Dec 31 (Reuters) - The Nigerian nai...",.NGSEINDEX NGN=D1 NGFXR=ECI,STX FRX HOT GVD BANK MCE ECI EMRG AFR MKTREP N...
46,12/31/2015 15:14:46,BUZZ-Amira Nature Foods Ltd: Wins $92 mln cont...,** Specialty rice packager's shares <ANFI....,ANFI.N,STX HOT NCYC FOBE FOTB CYCS CCOS US CA LEN BLR...
51,12/31/2015 15:15:30,BUZZ-WPX Energy Inc: Sells certain assets for ...,** Oil and gas company's shares <WPX.N> up...,WPX.N,STX COM FUND HOT PVE DRIL OILQ FINS BISV INVS ...
53,12/31/2015 15:18:01,Reuters Insider - UK regulator ditches banking...,Click the following link to watch video: ...,BARC.L HSBA.L PMR.L,AMERS ASIA BANK BISV BSVC CMPNY CN ECI EMRG EU...


In [4]:
##Index 생성
sample_data["Story_num"] = sample_data.index
sample_data

,Story Date Time,Headline,Story Body,Related RICs,Topics,Story_num
1,12/31/2015 15:00:00,"TABLE-U.S. 30-year, 15-year mortgages",DEC 31 (REUTERS) - FREDDIE MAC <FMCC.OB> ...,FMCC.OB,FIN US COFS MTG INT DBT WASH AGN BSVC FINS MCE...,1
2,12/31/2015 15:00:03,US STOCKS-Indexes open lower on last day of year,* S&P 500 turns negative for year * Ap...,.DJI .INX .IXIC .SPX,STX US MKTREP REP MTGFX LEN AMERS RTRS,2
13,12/31/2015 15:01:27,UPDATE 2-Brazil's Itaú buys BTG Pactual's debt...,"(Adds quote, background) By Guillermo Par...",BBTG11.SA ITUB4.SA BPAC3.SA ITUB.N,LOA CDM FINS BANK INVS SFIN DFIN INVD DEAL1 EM...,13
27,12/31/2015 15:04:09,UPDATE 1-Saudi's market regulator refers suspe...,* CMA refers unnamed suspects to authoriti...,7020.SE,TCOM PHON WLES ACCI EMRG SA MEAST JUDIC FRAUD1...,27
38,12/31/2015 15:08:13,BUZZ-Prima BioMed Ltd: Roth Capital starts cov...,** Australian biotech's U.S.-listed shares...,PRR.AX,STX HOT HECA PHMR RCH US LEN BLR HEA CMPNY MRC...,38
40,12/31/2015 15:08:39,BUZZ-MXN-Banxico rate hike vote was unanimous,* Decision to hike rates 25bps (to 3.2...,LAN.SN,AIRL AIRS AMERS CL CMPNY EMRG FRX INDS INSI LA...,40
43,12/31/2015 15:10:13,"Nigerian naira down 10 pct yr/yr, stocks fall ...","LAGOS, Dec 31 (Reuters) - The Nigerian nai...",.NGSEINDEX NGN=D1 NGFXR=ECI,STX FRX HOT GVD BANK MCE ECI EMRG AFR MKTREP N...,43
46,12/31/2015 15:14:46,BUZZ-Amira Nature Foods Ltd: Wins $92 mln cont...,** Specialty rice packager's shares <ANFI....,ANFI.N,STX HOT NCYC FOBE FOTB CYCS CCOS US CA LEN BLR...,46
51,12/31/2015 15:15:30,BUZZ-WPX Energy Inc: Sells certain assets for ...,** Oil and gas company's shares <WPX.N> up...,WPX.N,STX COM FUND HOT PVE DRIL OILQ FINS BISV INVS ...,51
53,12/31/2015 15:18:01,Reuters Insider - UK regulator ditches banking...,Click the following link to watch video: ...,BARC.L HSBA.L PMR.L,AMERS ASIA BANK BISV BSVC CMPNY CN ECI EMRG EU...,53


In [38]:
# Topic List 읽어오기
topic_data = pd.read_csv('../../02. Data/Reuter_topic_codes_Business.csv', encoding ='utf-8', sep = ",", names = ['topics'],header = None)
topic_data

,topics
0,ABS
1,AGN
2,BQ1
3,CDM
4,CDO
5,CDV
6,COM
7,DBT
8,DRV
9,EQB


In [41]:
# Story Body Random 확인
print(sample_data["Story Body"].iloc[1])

    * S&P 500 turns negative for year     * Apple weighs on S&P 500, Nasdaq; McDonald's drags Dow     * Energy sector worst performer on S&P 500 for year     * Indexes down: Dow 0.57 pct, S&P 0.49 pct, Nasdaq 0.46 pct   (Updates to open)     By Abhiram Nandakumar     Dec 31 (Reuters) - U.S. stock indexes were lower on Thursday morning, with the S&P 500 slipping back into negative territory for the year, as crude logged in a second year of steep losses.     The rout in commodities has sent markets across the world reeling as oil prices were hit by an unprecedented global glut that may take another year to clear. [O/R]     Shares of Chevron <CVX.N> were off 1 percent at $89.21, while Exxon <XOM.N> was down 0.7 percent at $77.61.     Energy stocks have taken a beating this year, with the S&P energy sector <.SPNY> losing nearly 24 percent this year, followed by a near 10 percent loss in materials <.SPLRCM>.     The S&P 500 was down 0.3 percent for the year and the Dow Jones industrial aver

## 2. Preprocessing

In [28]:
# 1. Stopword 목록 읽어오기 (stopword 목록 다운로드 -> excel로 내려받아 추후 수정 필요***)
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english') + ["n't", "'s", "'m", "ca"] + list(ENGLISH_STOP_WORDS))
stop_words_edit = pd.DataFrame(data = {"word":  list(stop_words)})
stop_words_edit.to_csv("../../02. Data/Stopword.csv", sep=',', index = False)

[nltk_data] Downloading package stopwords to /Users/dam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
# 2. 특수문자 목록 읽어오기
translator = list(str.maketrans('', '', string.punctuation)) + ["-----", "---", "...", "“", "”", "'ve"]

In [30]:
# 3. 실제 데이터 전처리 수행 (code 읽는 방법 : in 기준으로 읽고, for 기준으로 읽어들이는 단위 일치화 시키기)

# 3.1) each.translate(translator) == 특수문자 제거
# 3.2) x.lower() == 소문자화
# 3.3) if x.lower() not in stop_words == 불용어제거
# 3.4) if x.lower().isalpha() == 숫자제거
# 3.5) 동사 시제 제거 (lemmatization or stemming)
# 3.6) Basic 토크나이저
# 3.7) 광고 및 e-mail 제거

# 3.2, 3.3, 3.5, 3.6 수행
class myTokenizer:
    def __init__(self, parser):
        self.parser = parser
        
    def __call__(self, sample):
        # get the tokens using spaCy
        tokens = self.parser(sample)

        # lemmatize
        remove_pos = ['-PRON-','SPACE','ADP','PUNCT','DET', 'PART']

        lemmas = []
        for tok in tokens:
            # stopword가 아니고, remove_pos list에 없는 경우에만 lemma에 추가
            if (tok.lemma_.lower().strip() not in stop_words and translator) and str(tok.pos_) not in remove_pos : 
                item = str(tok.lemma_.lower().strip())+ "/" + str(tok.pos_)
                lemmas.append(item)

        tokens = lemmas

        return tokens

In [42]:
def cleanText(doc):
    ##문장 나누기
    doc = list(filter(None,doc.split('  ')))
    
    sentences = []
    
    # 3.7 수행 구역
    for text in doc:
         ##광고 제거
        text = text.split("<^")[0]
    
        ##괄호 문장 제거 [] ()
        #text = remove_text_inside_brackets(text)
        text = regex.sub(r'\([^()]*+(?:(?R)[^()]*)*+\)', '', text)
        text = re.sub("[\<\(\[].*?[\)\]\>]", "", text)
    
        ## E-mail 제거
        email_regex = re.compile(r'(?i)\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b')
        text= email_regex.sub('', text)
        
        ##HyperLink 제거
        text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
        
        # replace HTML symbols 
        text = text.replace("&amp;", "and").replace("&gt;", ">").replace("&lt;", "<")
        
        ##특수문자 제거
        text = re.sub("[^.,\'\"a-zA-Z ]",' ', text)
        
        ##양 끝 띄어쓰기 제거
        text = text.strip()
        
        if len(text) < 2 or text ==",":
            next
        ##담기.
        sentences.append(text)

    return list(filter(None,sentences))

def remove_one(text):
    remove_start_line = []
    remove_start_line.append("For the full text of this story please click the following link")
    remove_start_line.append("Click the following link to watch video:")
    remove_start_line.append("Keywords:")
    remove_start_line.append("Further company coverage:")
    remove_start_line.append("Source text for")
    remove_start_line.append("Short Link:")
    remove_start_line.append("Source:")
    remove_start_line.append("Description:")
    
    for item in remove_start_line:
        if item in text:
            return True
    
    return False

# def my_tokenizer(text):
#     temp = text.split('  - ')
#     temp = "  ".join(temp)
#     temp = temp.split('  * ')
#     temp = "  ".join(temp)
#     temp = temp.split('  ')
#     sentences = []
#     for sentence in temp:
#         item = sentence.strip()
#         if not item is None and not item == "" and not remove_one(sentence):
#             sentences.append(item)
            
#     return sentences

In [43]:
# Story Line Column 한 통에 넣기
sentences = []
for doc in sample_data['Story Body'].tolist():
    sentences.extend(cleanText(doc))
print(sentences[:100]) # 데이터 확인용

['DEC       FREDDIE MAC  AVERAGE U.S. MORTGAGE RATES  FOR WEEKS ENDING', 'DEC', 'DEC', 'YEAR AGO', 'YR', '.', '.', '.', 'YR', '.', '.', '.', 'YR ARM', '.', '.', '.', 'YR ARM', '.', '.', '.', 'U.S.', 'NORTH', 'SOUTH', 'NORTH', 'SOUTH', 'WEST', 'AVG.', 'EAST', 'EAST', 'CENTRAL', 'WEST', 'YR', '.', '.', '.', '.', '.', '.', 'FEES PTS', '.', '.', '.', '.', '.', '.', 'YR', '.', '.', '.', '.', '.', '.', 'FEES PTS', '.', '.', '.', '.', '.', '.', 'YR ARM', '.', '.', '.', '.', '.', '.', 'FEES PTS', '.', '.', '.', '.', '.', '.', 'YR ARM', '.', '.', '.', '.', '.', '.', 'FEES PTS', '.', '.', '.', '.', '.', '.', 'S P     turns negative for year', "Apple weighs on S P    , Nasdaq  McDonald's drags Dow", 'Energy sector worst performer on S P     for year', 'Indexes down  Dow  .   pct, S P  .   pct, Nasdaq  .   pct', 'By Abhiram Nandakumar', 'Dec       U.S. stock indexes were lower on Thursday morning, with the S P     slipping back into negative territory for the year, as crude logged in a second year

In [ ]:
# 3.6 tokenizer 사용하여 Token단위 DF 생성
col_names = ['word', 'pos']
word = []
pos = []

parser = en_core_web_sm.load()
basic_tokenizer = myTokenizer(parser)
for i in range(len(sentences)) : 
# for i in range(5): 
    for j in range(len(basic_tokenizer(sentences[i]))):
        if i == 0 and j==0 :
            tokens = pd.DataFrame([basic_tokenizer(sentences[i])[j].split("/")], columns = col_names)
        else :
            tokens = tokens.append(pd.DataFrame([basic_tokenizer(sentences[i])[j].split("/")], columns =col_names), ignore_index=True)
tokens